# Build a reduced ETC model from RECON3D

In [1]:
from pytfa.redgem.lumpgem import LumpGEM
from pytfa.io.json import load_json_model
from cobra import Reaction
from cobra.util import Zero

#recon3d = load_json_model('./../data/GEM_Recon3_thermo_genes.json')
recon3d = load_json_model('./../data/GEM_Recon3_thermo_genes_cleaned_constraints.json')

recon3d.solver.configuration.tolerances.feasibility = 1e-9
recon3d.solver.configuration.tolerances.optimality = 1e-9 

# Resting membrane potential
recon3d.compartments['c']['membranePot']['e'] = 60
recon3d.compartments['e']['membranePot']['c'] = -60
# Resting mitochondrial membrane potential
recon3d.compartments['m']['membranePot']['c'] = 150
recon3d.compartments['c']['membranePot']['m'] = -150

recon3d.compartments['i']['pH'] = 7.3 # 0.1 more than cytosol

recon3d.repair()


2024-09-23 13:10:32,960 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


In [ ]:
# Thermodynamic annotation of Sodium and Potassium
recon3d.thermo_data['metabolites']['cpd00205']['deltaGf_std'] = 1.0
recon3d.thermo_data['metabolites']['cpd00205']['deltaGf_err'] = 0.1
recon3d.thermo_data['metabolites']['cpd00205']['error']  = 'Nil'
recon3d.thermo_data['metabolites']['cpd00205']['pKa']  = []
recon3d.thermo_data['metabolites']['cpd00971']['pKa']  = []

In [ ]:
# Clean constraints and variables to rebuild thermo 

# from tqdm import tqdm
# from pytfa.optim.constraints import ReactionConstraint
# from pytfa.optim.variables import ReactionVariable
# from pytfa.optim.utils import get_all_subclasses

# all_cons_subclasses = get_all_subclasses(ReactionConstraint)
# all_var_subclasses = get_all_subclasses(ReactionVariable)


# # remove all thermo variables and constratines 
# for var_subclass in all_var_subclasses:
#     var_subclass = recon3d._var_kinds[var_subclass.__name__]
#     for var in tqdm(var_subclass):
#         recon3d.remove_variable(var)

# for cons_subclass in all_cons_subclasses:
#     cons_subclass = recon3d._cons_kinds[cons_subclass.__name__]
#     for cons in tqdm(cons_subclass):
#         recon3d.remove_constraint(cons)

# recon3d.repair()

# from pytfa.io.json import save_json_model
# save_json_model(recon3d, './../data/GEM_Recon3_thermo_genes_cleaned_constraints.json')


In [ ]:
######################################################
# Curate the electron transport chain thermodynamics #
######################################################


# 1kcal = 4.184e3 J
F = 96485/4.184e3 # Faraday constant in kcal/V

# Cytochrome redox pair 
# cyt c (Fe3+) + e- -> cyt c (Fe2+)
dE0_cytochrome = 0.25 # V
n = 1
dG = -n*F*dE0_cytochrome

dG0_Fe2  = -18.85 # kJ/mol
dG0_Fe3  = dG0_Fe2 - dG

# Add an entry for the cytochrome c Fe2+
recon3d.compounds_data['cpd00110'] = recon3d.compounds_data['cpd00109'].copy()
recon3d.compounds_data['cpd00110']['id'] = 'cpd00110'
recon3d.compounds_data['cpd00110']['name'] = 'Cytochrome c Fe2+'
recon3d.compounds_data['cpd00110']['struct_cues'] = {'Fe2': 1}
recon3d.compounds_data['cpd00110']['deltaGf_std'] = dG0_Fe2
recon3d.compounds_data['cpd00110']['deltaGf_err'] = 0.5
recon3d.compounds_data['cpd00110']['charge_std'] = 0
recon3d.compounds_data['cpd00110']['formula'] = 'C42H54FeN8O6S2'
recon3d.compounds_data['cpd00110']['mass_std'] = 884
recon3d.compounds_data['cpd00110']['pKa'] = []

# Add an entry for the cytochrome c Fe3+
recon3d.compounds_data['cpd00109']['id'] = 'cpd00109'
recon3d.compounds_data['cpd00109']['name'] = 'Cytochrome c Fe3+'
recon3d.compounds_data['cpd00109']['struct_cues'] = {'Fe3': 1}
recon3d.compounds_data['cpd00109']['deltaGf_std'] = dG0_Fe3
recon3d.compounds_data['cpd00109']['deltaGf_err'] = 0.5
recon3d.compounds_data['cpd00109']['charge_std'] = 1
recon3d.compounds_data['cpd00109']['formula'] = 'C42H54FeN8O6S2'
recon3d.compounds_data['cpd00109']['mass_std'] = 884
recon3d.compounds_data['cpd00109']['pKa'] = []

# Flavin redox pair (is this the right pair?)
# FAD + 2H+ + 2e- -> FADH2 (currently FAD is deprotonated so missing an H+)
E0_flavin = -0.219 # V
n = 2 
dG = -n*F*E0_flavin
dG_H = recon3d.metabolites.h_m.thermo['deltaGf_std']

dG0_FAD  = -229.75 # kJ/mol
dG0_FADH2  = dG0_FAD - dG + dG_H # Account for the additional H+ in the reaction


# FAD https://modelseed.org/biochem/compounds/cpd00015
fad_data = recon3d.metabolites.fad_m.thermo.__dict__
fad_data['deltaGf_std'] = dG0_FAD
fad_data['deltaGf_err'] = 2.22
fad_data['mass_std'] = recon3d.metabolites.fad_m.thermo['mass']
recon3d.compounds_data['cpd00015'].update(fad_data) # Update the compound databse 

# FADH2 https://modelseed.org/biochem/compounds/cpd00982 # Database entry C01352
fadh2_data = recon3d.metabolites.fadh2_m.thermo.__dict__
fadh2_data['deltaGf_std'] = dG0_FADH2
fadh2_data['deltaGf_err'] = 2.22
fadh2_data['mass_std'] = recon3d.metabolites.fadh2_m.thermo['mass']
recon3d.compounds_data['C01352'].update(fadh2_data) # Update the compound databse



In [ ]:
# annotate ficytC_m
recon3d.metabolites.ficytC_m.annotation['seed_id'] = 'cpd00109' # This is the Fe3+ form
recon3d.metabolites.focytC_m.annotation['seed_id'] = 'cpd00110' # This is the Fe2+ form

recon3d.metabolites.ficytC_m.charge = 1 #2
recon3d.metabolites.focytC_m.charge = 0 #3

# Model Electron transfer flavoprotein oxidized/redcued
# etfox_m as fad_m and etfrd_m as fadh2_m
recon3d.metabolites.etfox_m.annotation['seed_id'] = 'cpd00015'
recon3d.metabolites.etfrd_m.annotation['seed_id'] = 'C01352'


In [ ]:
# Thermo for sodium
recon3d.compounds_data['cpd00971']['deltaGf_std'] = 1.0
recon3d.compounds_data['cpd00971']['deltaGf_err'] = 0.1
recon3d.compounds_data['cpd00971']['error']  = 'Nil'

In [ ]:
###########################################
## Prepare the model lumpGEM
###########################################


# Add dummy reactions for ATP hydrolysis
cyt_atp2adp = Reaction('cyt_atp2adp')
recon3d.add_reactions([ cyt_atp2adp,])
cyt_atp2adp.reaction = 'atp_c + h2o_c --> adp_c + pi_c + h_c'

# ADD Phosphate translocase reaction (PiC)
# TODO: What is the correct stoichiometry for this reaction?
PiC = Reaction('PiC')
recon3d.add_reactions([ PiC,])
PiC.reaction = 'pi_c + h_c -> pi_m + h_m'

# # Add proton translocase reaction (H+)
# Htic = Reaction('Htic')
# recon3d.add_reactions([ Htic,])
# Htic.reaction = 'h_i <=> h_c'

# This converts cobra reactions to pytfa (adds binary variables/constrains)
recon3d.objective = Zero # Symbolic zero objective

recon3d.prepare()
recon3d.convert()


2024-08-16 13:31:49,327 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...
2024-08-16 13:32:04,219 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/F
2024-08-16 13:32:04,220 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/U
2024-08-16 13:32:04,222 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-08-16 13:32:04,224 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-08-16 13:32:04,226 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/F
2024-08-16 13:32:04,228 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/U
2024-08-16 13:32:04,231 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-08-16 13:32:04,232 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-08-16 13:32:04,268 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52H9

In [ ]:
[(r.id, r.reaction) for r in recon3d.metabolites.mal_L_c.reactions]

[('r2391', 'HC00342_c + h_c + mal_L_m --> HC00342_m + h_m + mal_L_c'),
 ('r0913', 'icit_m + mal_L_c <=> icit_c + mal_L_m'),
 ('MAL_Ltx', 'mal_L_x <=> mal_L_c'),
 ('MALOAtm', 'mal_L_m + oaa_c <=> mal_L_c + oaa_m'),
 ('MALtm', 'mal_L_c + pi_m <=> mal_L_m + pi_c'),
 ('ME2', 'mal_L_c + nadp_c --> co2_c + h_c + nadph_c + pyr_c'),
 ('MALTSULtm', 'mal_L_c + tsul_m <=> mal_L_m + tsul_c'),
 ('HMR_4964', 'cit_c + h_c + mal_L_m --> cit_m + h_m + mal_L_c'),
 ('r0822', 'fum_c + mal_L_m <=> fum_m + mal_L_c'),
 ('MDH', 'mal_L_c + nad_c <=> 2.0 h_c + nadh_c + oaa_c'),
 ('AKGMALtm', 'akg_m + mal_L_c <=> akg_c + mal_L_m'),
 ('MAL_Lte', 'mal_L_e <=> mal_L_c'),
 ('CITtam', 'cit_c + mal_L_m <=> cit_m + mal_L_c'),
 ('r2387', 'h_c + icit_c + mal_L_m --> h_m + icit_m + mal_L_c'),
 ('MALSO4tm', 'mal_L_c + so4_m <=> mal_L_m + so4_c'),
 ('r2379', 'HC00342_m + mal_L_c <=> HC00342_c + mal_L_m'),
 ('FUM', 'fum_c + h2o_c <=> mal_L_c'),
 ('MALSO3tm', 'mal_L_c + so3_m <=> mal_L_m + so3_c')]

In [ ]:
# Define the core model as all reactions that connect the ETC intermediates (RedGEM, paper = D0 ) 
# Quinones, Cytochromes, and ATP synthase

ubiquinone_reqations = [r.id for r in recon3d.metabolites.q10_m.reactions if recon3d.metabolites.q10h2_m in r.metabolites]
ferrocytochrome_reactions = [r.id for r in recon3d.metabolites.focytC_m.reactions if recon3d.metabolites.ficytC_m in r.metabolites]

# Malate aspartate shuttle 
aspartate_malate_shuttle = ['MDH', 'ASPTA', 'ASPTAm', 'MDHm', 'ASPGLUm', 'AKGMALtm']

# Glycerol 3-phosphate shuttle muscle
glycerol_3_phosphate_shuttle = ['G3PD1','r0205']

# Lactate shuttle muscle
lactate_shuttle = ['LDH_L', 'LDH_Lm','L_LACtm','PYRt2m']

core_reaction_ids = list(set(ubiquinone_reqations + ferrocytochrome_reactions + ['ATPS4mi'] + glycerol_3_phosphate_shuttle + lactate_shuttle + aspartate_malate_shuttle))
core_reactions = [r for r in recon3d.reactions if r.id in core_reaction_ids]

# Print a table with the core reactions
print('Core reactions')
print('--------------')
for r in core_reactions:
    print(r.id, r.reaction, r.thermo)



Core reactions
--------------
AKGMALtm akg_m + mal_L_c <=> akg_c + mal_L_m {'isTrans': True, 'computed': True, 'deltaGR': 0.0012994733051527874, 'deltaGrxn': 0, 'deltaGRerr': 2}
ASPGLUm asp_L_m + glu_L_c + h_c --> asp_L_c + glu_L_m + h_m {'isTrans': True, 'computed': True, 'deltaGR': -5.242772133284354, 'deltaGrxn': 0, 'deltaGRerr': 2}
ASPTAm akg_m + asp_L_m <=> glu_L_m + oaa_m {'isTrans': False, 'computed': True, 'deltaGR': -1.144983579865368, 'deltaGRerr': 2}
DHORD9 dhor_S_c + q10_m --> orot_c + q10h2_m {'isTrans': False, 'computed': True, 'deltaGR': -10.478621558705754, 'deltaGRerr': 2.3526057893323307}
ETFQO etfrd_m + q10_m --> etfox_m + h_m + q10h2_m {'isTrans': False, 'computed': True, 'deltaGR': -13.419995909336194, 'deltaGRerr': 3.48528908413635}
LDH_Lm lac_L_m + nad_m <=> 2.0 h_m + nadh_m + pyr_m {'isTrans': False, 'computed': True, 'deltaGR': -7.262871409359349, 'deltaGRerr': 1.6149334351607187}
L_LACDcm 2.0 ficytC_m + lac_L_c --> 2.0 focytC_m + 2.0 h_c + pyr_c {'isTrans': Fa

In [ ]:
# Refine Glycolysis corrected delta G constraints from Lehninger:
# Units are in kJ/mol
glycolysis_thermodynamics = {
    'HEX1': -16.7,
    'PGI': 1.7,
    'PFK': -14.9,
    'FBA': 23.8,
    'TPI': 5.6, # Equilibrator value
    'GAPD': 6.3,
    'PGK': 18.8,
    'PGM': 4.4, 
    'ENO': 7.5,
    'PYK': -31.4,
    'LDH_L': 25.1,
    'LDH_Lm': 33.1,
    }

# Converstion to kcal/mol
scaling = 0.239006
for r_id, dg in glycolysis_thermodynamics.items():
    dgo = recon3d.delta_gstd.get_by_id(r_id).variable
    try:
        dgo.ub = dg * scaling + 0.5
        dgo.lb = dg * scaling - 0.5
    except ValueError:
        dgo.lb = dg * scaling - 0.5
        dgo.ub = dg * scaling + 0.5

    recon3d.optimize()
    print(f"{r_id} : {dgo.lb} < {dgo.primal} < {dgo.ub} kcal/mol")
    

sol = recon3d.optimize()

HEX1 : -4.491400199999999 < -3.4914001999999997 < -3.4914001999999997 kcal/mol
PGI : -0.09368979999999999 < -0.09368979999999999 < 0.9063102000000001 kcal/mol
PFK : -4.0611894 < -4.0611894 < -3.0611894 kcal/mol
FBA : 5.1883428 < 5.1883428 < 6.1883428 kcal/mol
TPI : 1.292545 < 1.292545 < 2.292545 kcal/mol
GAPD : 1.0057378 < 1.0057378 < 2.0057378 kcal/mol
PGK : -4.9933128 < -4.9933128 < -3.9933128 kcal/mol
PGM : 0.5516264 < 0.5516264 < 1.5516264 kcal/mol
ENO : 1.292545 < 1.292545 < 2.292545 kcal/mol
PYK : -8.004788399999999 < -8.004788399999999 < -7.0047884 kcal/mol
LDH_L : 5.4990506 < 5.4990506 < 6.4990506 kcal/mol
LDH_Lm : 7.4110986 < 7.4110986 < 8.411098599999999 kcal/mol


In [ ]:
# Refine TCA corrected delta G constraints from Lehninger:
# Units are in kJ/mol
tca_thermodynamics = {
    'PDHm' : -33.4,
    'CSm' : -32.3,
    'ACONTm' : 13.3,
    'ICDHxm' : 10.0, # Equilibrator value ?!?!
    'AKGDm' : -33.5,
    'SUCOAS1m' : -2.9,
    #'r0509' : 0, 
    'FUMm': -3.8,
    'MDHm' : 29.7,
    'MDH': 29.7 + 2.3 * 2 * 310 * 2.26 * (0.8) /1000 , # MDH delta proton correction in cytosol (nH 2.3 RT * delta pH)
    'ADK1': 0.3, # Equilibrator value ?!
    'NDPK1m': -2.7, # Equilibrator value  (EC: 2.7.4.6)
    'FADH2ETC': -70/2, # Equilibrator  value ?
    'G3PD1': 22.6, # Equilibrator value ?!?
    'r0205': -59.7, # Equilibrator value ?!?
    'PiC': 2.3 * 310 * 2.26 * (-0.8) / 1000 ,  # Charge NEUTRAL transport of phophate -> only proton force (nH 2.3 RT * delta pH)
    'PPA': -19, # 
    'FACOAL160i': -15, # Lehninge
    'BDHm': 8.0, # Equilibrator value (EC: 1.1.1.30)
    'OCOAT1m': 12.6, # Equilibrator value (EC: 2.8.3.5)
    'ACACT1rm': 25, # Equilibrator value (EC: 2.3.1.9)
    'C160CPT1': -2.2, # Equilibrator value (EC: 2.3.1.21 reverse)
    'C160CPT1': 2.2, # Equilibrator value (EC: 2.3.1.21)
}

# Converstion to kcal/mol
scaling = 0.239006
for r_id, dg in tca_thermodynamics.items():
    dgo = recon3d.delta_gstd.get_by_id(r_id).variable
    try:
        dgo.ub = dg * scaling + 0.5
        dgo.lb = dg * scaling - 0.5
    except ValueError:
        dgo.lb = dg * scaling - 0.5
        dgo.ub = dg * scaling + 0.5

    recon3d.optimize()
    print(f"{r_id} : {dgo.lb} < {dgo.primal} < {dgo.ub} kcal/mol")
    

sol = recon3d.optimize()

PDHm : -8.482800399999999 < -8.482800399999999 < -7.4828003999999995 kcal/mol
CSm : -8.2198938 < -8.2198938 < -7.2198937999999995 kcal/mol
ACONTm : 2.6787798 < 2.6787798 < 3.6787798 kcal/mol
ICDHxm : 1.89006 < 1.89006 < 2.89006 kcal/mol
AKGDm : -8.506701 < -8.506701 < -7.506701 kcal/mol
SUCOAS1m : 0.1931174 < 0.1931174 < 1.1931174 kcal/mol
FUMm : -1.4082227999999999 < -0.4082228 < -0.4082228 kcal/mol
MDHm : 6.5984782 < 6.5984782 < 7.5984782 kcal/mol
ADK1 : -0.4282982 < -0.4282982 < 0.5717018 kcal/mol
FADH2ETC : -8.86521 < -8.86521 < -7.865209999999999 kcal/mol


In [ ]:
# Save the reactions bounds from the tissue specific model
tissue_reaction_ko = dict()
for reaction in recon3d.reactions:
    if reaction.lower_bound==0 and reaction.upper_bound==0:
        tissue_reaction_ko[reaction.id] = (reaction.lower_bound, reaction.upper_bound)

In [ ]:
# Sanity check
# Compute the redox potential of the Q10/Q10H2 couple in the model
# Q10 + 3H+ + 2e- -> Q10H2 
dG = recon3d.metabolites.fad_m.thermo['deltaGf_std'] + 3*recon3d.metabolites.h_m.thermo['deltaGf_std'] - recon3d.metabolites.fadh2_m.thermo['deltaGf_std']
# Convert to V
# ΔG=−nFE 
# 1kcal = 4.184e3 J
F = 96485/4.184e3
n = 2

E = dG / (n * F)
E # should be arround -0.22 V

-0.19426133595895678

In [ ]:
# For future reference making debuging easier
#recon3d.solver.problem.conflict.refine()
#recon3d.solver.problem.conflict.write('bla.txt')

In [ ]:
# Make sure that transporters dont have unrealistic equilibrium constants
EPSILON_DG = 1e-6

exceptions = ['ATPS4mi','CYOR_u10mi','CYOOm2i','NADH2_u10mi','PiC',]
# Print transport deltaG0
for r in recon3d.reactions:
    if r.thermo['isTrans'] and (r.id not in exceptions):
        try:
            print(f"{r.id} : {r.reaction} {r.thermo['deltaGR']} kcal/mol")
            dgo = recon3d.delta_gstd.get_by_id(r.id)
            dgo.variable.ub = r.thermo['deltaGR'] + EPSILON_DG
            dgo.variable.lb = r.thermo['deltaGR'] - EPSILON_DG
        except KeyError:
            pass


recon3d.optimize()


In [ ]:
recon3d.reactions.get_by_id('cyt_atp2adp').bounds = (1, 200)
recon3d.reactions.get_by_id('PiC').bounds = (0, 400)

In [ ]:
# Open bounds from -100 -> -200 and 100 to 200
for r in recon3d.reactions:
    if r.bounds[0] == -100:
        r.lower_bound = -200
    if r.bounds[1] == 100:
        r.upper_bound = 200 

In [ ]:
# By default only allow secretions for lumping + defined medium
for rxn in recon3d.boundary:
    rxn.bounds = (0, 200)

# Allow for protons exchange and oxygen uptake
recon3d.reactions.get_by_id('EX_h_e').bounds = (-200.0, 200.0)
recon3d.reactions.get_by_id('EX_h2o_e').bounds = (-200.0, 200.0)
recon3d.reactions.get_by_id('EX_o2_e').bounds = (-200.0, 0.0)

# Lactate and pyruvate transporters should be reversible
recon3d.reactions.get_by_id('L_LACtm').bounds = (-100, 100)
recon3d.reactions.get_by_id('PYRt2m').bounds = (-100, 100)


# Test the model (For lumpting close all carbon sources)
recon3d.reactions.get_by_id('EX_lac_L_e').bounds = (0, 100)
recon3d.reactions.get_by_id('EX_glc_D_e').bounds = (0, 100)
recon3d.reactions.get_by_id('EX_bhb_e').bounds = (0, 100)
recon3d.reactions.get_by_id('EX_hdca_e').bounds = (-1, 100)

recon3d.reactions.get_by_id('ATPS4mi').bounds = (1,200)
recon3d.reactions.get_by_id('ATPtm').bounds = (-200,200)
# These are some FA reaction that can cause weird cycling
recon3d.reactions.r0310.bounds = (0, 0)
recon3d.reactions.HMR_3121.bounds = (0, 0)

# This can bypass the Malate shuttle should not
recon3d.reactions.MALOAtm.bounds = (0, 0)   

# Remove the rxn with the weired stoich 
recon3d.reactions.CYOOm3i.bounds = (0, 0)

# Allow this reaction to be reversible (in reality this runs backwards)
# See Lehninger Principles of Biochemistry
recon3d.reactions.ACACT1rm.bounds = (-200,200)

# Block the force reaction to bo forward (this should not make ATP)
# https://www.proteinatlas.org/ENSG00000106992-AK1/tissue+cell+type
recon3d.reactions.ADK1.bounds = (0,100) # THIS IS THE MAIN ADK expressed in muscle (AK1 and AK2) 

# This is an entorcyte specific transport not active in muscle
recon3d.reactions.PALFATPtc.bounds = (0,0)

# Block nadph reactions (erros in beta ox)
for r in recon3d.metabolites.nadp_m.reactions:
    r.bounds = (0,0)
for r in recon3d.metabolites.nadp_c.reactions:
    r.bounds = (0,0)

# Test w/o direct ATP production in SUCOASm
recon3d.reactions.SUCOASm.bounds = (0,0)

# Relax dGo of NADH2_u10mi 
# wrong calculation since not 6 but only 4 protons are transported across the membrane
# the other ones are stashed onto qh2 
dg = -3.957108259354982 + (6.544013627216948 + 24.905879999999996) * 4/6 

# From this publication 
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2931722/
# The standard Gibbs free energy change for the reaction is -111 kJ/mol
dg =  -111* 0.24 + (6.544013627216948 + 24.905879999999996) * 4/6 

recon3d.delta_gstd.NADH2_u10mi.variable.lb = dg - 5
recon3d.delta_gstd.NADH2_u10mi.variable.ub = dg + 5


recon3d.objective = recon3d.reactions.get_by_id('cyt_atp2adp')

sol = recon3d.optimize()
print(sol)


<Solution 106.000 at 0x1ee8e4e52e0>


In [ ]:
import numpy as np
from pytfa.optim.constraints import ModelConstraint

# Force ATP/ADP ratio to be 10
# Cytosolic ATP/ADP ratio
# https://pubmed.ncbi.nlm.nih.gov/749453/



expression = recon3d.log_concentration.atp_c.variable - recon3d.log_concentration.adp_c.variable
id_='atp_c_adp_c_ratio'

recon3d.add_constraint(ModelConstraint,
                        hook=recon3d,
                        expr=expression,
                        id_=id_, 
                        lb=np.log(10), 
                        ub=np.log(100),
                        )
print(f"Integrate cytosolic ATP/ADP ratio: 10-50 ")
sol = recon3d.optimize()
print(f"ATP yield: {sol.objective_value}")


# Force ATP/ADP ratio to bearround 2 and 8
# Cytosolic ATP/ADP ratio
# https://www.mdpi.com/1422-0067/23/10/5550


expression = recon3d.log_concentration.atp_m.variable - recon3d.log_concentration.adp_m.variable
id_='atp_m_adp_m_ratio'

recon3d.add_constraint(ModelConstraint,
                        hook=recon3d,
                        expr=expression,
                        id_=id_, 
                        lb=np.log(2), 
                        ub=np.log(8),
                        )
print(f"Integrate mitochondrial ATP/ADP ratio: 2-8")
sol = recon3d.optimize()
sol = recon3d.optimize()
print(f"ATP yield: {sol.objective_value}")

# mitochondrial GTP/GDP ratio
expression = recon3d.log_concentration.gtp_m.variable - recon3d.log_concentration.gdp_m.variable

recon3d.add_constraint(ModelConstraint,
                        hook=recon3d,
                        expr=expression,
                        id_='gtp_m_gdp_m_ratio', 
                        lb=np.log(10), 
                        ub=np.log(300),
                        )
print(f"Integrate mitochondrial GTP/GDP ratio: 10-300")
recon3d.optimize()
sol = recon3d.optimize()
print(f"ATP yield: {sol.objective_value}")        

# mitochondrial NAD/NADH ratio
expression = recon3d.log_concentration.nad_m.variable - recon3d.log_concentration.nadh_m.variable

recon3d.add_constraint(ModelConstraint,
                        hook=recon3d,
                        expr=expression,
                        id_='nad_m_nadh_m_ratio', 
                        lb=np.log(6), 
                        ub=np.log(8),
                        )
print(f"Integrate mitochondrial NAD+/NADH ratio: 6-8")
recon3d.optimize()
sol = recon3d.optimize()
print(f"ATP yield: {sol.objective_value}")  

# NOTE: This is currently not recomended gives stupid networks because of wrong deltag G values in glycolyis and TCA
# TODO: Add some manual curation of the dG values for the reactions in the recon3d model then rerun this

# # Cytoplasmic NAD/NADH ratio
expression = recon3d.log_concentration.nad_c.variable - recon3d.log_concentration.nadh_c.variable

recon3d.add_constraint(ModelConstraint,
                        hook=recon3d,
                        expr=expression,
                        id_='nad_c_nadh_c_ratio', 
                        lb=np.log(60), 
                        ub=np.log(700),
                        )

print(f"Integrate cytoplasmic NAD+/NADH ratio: 60-700")
recon3d.optimize()
sol = recon3d.optimize()
print(f"ATP yield: {sol.objective_value}")  

Integrate cytosolic ATP/ADP ratio: 10-50 
ATP yield: 106.0
Integrate mitochondrial ATP/ADP ratio: 2-8
ATP yield: 106.0
Integrate mitochondrial GTP/GDP ratio: 10-300
ATP yield: 106.0
Integrate mitochondrial NAD+/NADH ratio: 6-8
ATP yield: 106.0
Integrate cytoplasmic NAD+/NADH ratio: 60-700
ATP yield: 106.0


In [ ]:
# Print non-zero fluxes that are not transport reactions
for r in recon3d.reactions:
    if abs(sol.fluxes[r.id]) > 1e-6  and recon3d.metabolites.get_by_id('nadh_m') in r.metabolites:
        print(r.id, sol.fluxes[r.id], r.reaction, r.thermo)


AKGDm 8.0 akg_m + coa_m + nad_m --> co2_m + h_m + nadh_m + succoa_m {'isTrans': False, 'computed': True, 'deltaGR': -20.90829632014902, 'deltaGRerr': 1.9903323843016776}
FAOXC140 1.0 6.0 coa_m + 6.0 fad_m + 6.0 h2o_m + 6.0 nad_m + tdcoa_m --> 7.0 accoa_m + 6.0 fadh2_m + 6.0 h_m + 6.0 nadh_m {'isTrans': False, 'computed': True, 'deltaGR': -38.471374987856294, 'deltaGRerr': 16.342358243533887}
ICDHxm 8.0 icit_m + nad_m --> akg_m + co2_m + h_m + nadh_m {'isTrans': False, 'computed': True, 'deltaGR': -7.976421985260288, 'deltaGRerr': 1.7170517872213407}
MDHm 8.0 mal_L_m + nad_m <=> 2.0 h_m + nadh_m + oaa_m {'isTrans': False, 'computed': True, 'deltaGR': -7.262639713280066, 'deltaGRerr': 1.6149334351607187}
FAOXC16C14m 1.0 coa_m + fad_m + h2o_m + nad_m + pmtcoa_m --> accoa_m + fadh2_m + h_m + nadh_m + tdcoa_m {'isTrans': False, 'computed': True, 'deltaGR': -6.41189583130938, 'deltaGRerr': 2.7237263739223145}
NADH2_u10mi 31.0 6.0 h_m + nadh_m + q10_m --> 4.0 h_i + nad_m + q10h2_m {'isTrans':

In [ ]:
# Print non-zero fluxes that are not transport reactions
for r in recon3d.reactions:
    if  recon3d.metabolites.get_by_id('h_c') in r.metabolites and sol.fluxes[r.id] != 0:
        print(r.id, sol.fluxes[r.id], r.reaction, r.check_mass_balance())

ADK3 1.0 amp_c + gtp_c + h_c <=> adp_c + gdp_c {}
CYTK10 -2.333333333333333 cmp_c + dgtp_c + h_c <=> cdp_c + dgdp_c {}
CYTK12 -2.3333333333333335 dcmp_c + dctp_c + h_c <=> 2.0 dcdp_c {}
CYTK3 7.0 cmp_c + gtp_c + h_c <=> cdp_c + gdp_c {}
CYTK5 4.666666666666667 ctp_c + dcmp_c + h_c <=> cdp_c + dcdp_c {}
CYTK6 -4.666666666666667 cmp_c + ctp_c + h_c <=> 2.0 cdp_c {}
FACOAL160i 1.0 atp_c + coa_c + hdca_c --> amp_c + h_c + pmtcoa_c + ppi_c {}
RE1530C 2.3333333333333335 dgtp_c + duri_c <=> dgdp_c + dump_c + h_c {}
The 1.0000000000000049 h_e <=> h_c {}
PPA 1.0 h2o_c + ppi_c --> h_c + 2.0 pi_c {}
DCMPDA -2.3333333333333335 dcmp_c + h2o_c + h_c <=> dump_c + nh4_c {}
HMR_1095 2.3333333333333335 h_c <=> h_n {}
cyt_atp2adp 106.0 atp_c + h2o_c --> adp_c + h_c + pi_c {}
PiC 108.0 h_c + pi_c --> h_m + pi_m {}


In [ ]:
# recon3d.reactions.get_by_id('ATPS4mi').bounds = (0,200)
# recon3d.objective = recon3d.reactions.get_by_id('EX_lac_L_e')
# sol = recon3d.optimize()
# print(sol)
# # Constraint max lactate secretion
# # recon3d.reactions.get_by_id('EX_lac_L_e').bounds = (sol.objective_value, 100)

In [ ]:
from pytfa.io.json import save_json_model

save_json_model(recon3d, './../data/GEM_Recon3_Thermo_Lehninger_Curated.json')

In [ ]:
1/0

In [ ]:
# Test the model (For lumpting close all carbon sources)
recon3d.reactions.get_by_id('EX_lac_L_e').bounds = (0, 100)
recon3d.reactions.get_by_id('EX_glc_D_e').bounds = (0, 100)
recon3d.reactions.get_by_id('EX_bhb_e').bounds = (0, 100)
recon3d.reactions.get_by_id('EX_hdca_e').bounds = (0, 100)



In [ ]:
# Generate a lump for each carbon source 
EPSILON = 1e-6

# LumpGEM parameters
params = {
    'core_subsystems': [],
    'extracellular_system': [],
    'timeout': 3600,  # max time in s
    "constraint_method": 'both',
    # Stuff we dont need for this purpose
    "small_metabolites": [],
    "cofactor_pairs": [],
    "inorganics": [],
    "max_lumps_per_BBB": 10  # Maximal number of alternatives
}

In [ ]:
def find_lumps(carbon_source, n_carbon, model, params=params, 
                core_reactions=[], method='min',
                output='EX_co2_e', tissue_reaction_ko=None,
                max_energy_yield=0.9):  
        # Lump the reactions 
        
        params['growth_rate'] = n_carbon-EPSILON
        params['biomass_rxns'] =  [output,]

        resp_model = model.copy()

        # If tissue bounds are provided
        if not tissue_reaction_ko is None:
            for r_id, bounds in tissue_reaction_ko.items():
                resp_model.reactions.get_by_id(r_id).bounds = bounds
    
        resp_model.reactions.get_by_id(carbon_source).bounds = (-1-EPSILON, -1+EPSILON)
        resp_model.objective = resp_model.reactions.get_by_id(output)
        resp_model.objective_direction = 'max'
        sol = resp_model.optimize()
        print("Max CO2 production: ", sol.objective_value)

        # Maximal ATP production 
        resp_model.reactions.get_by_id(output).bounds = (n_carbon-EPSILON, n_carbon+EPSILON)
        resp_model.objective = resp_model.reactions.get_by_id('cyt_atp2adp')
        resp_model.objective_direction = 'max'
        sol = resp_model.optimize()
        print("Max ATP production: ", sol.objective_value)

        # Constraint max ATP production
        resp_model.reactions.get_by_id('cyt_atp2adp').bounds = (sol.objective_value*max_energy_yield-EPSILON*n_carbon, 200)

        # Reset CO2 constraint
        resp_model.reactions.get_by_id(output).bounds = (0, 100)

        resp_model.objective = Zero
        sol = resp_model.optimize()

        print("Test feasability: ", sol.objective_value)
        
        #LumpGEM needs to take as list of core reaction id as input
        subnetwork_extraction = LumpGEM(resp_model, core_reactions, params, min_transport=True, bigM=200 )
        lumps = subnetwork_extraction.compute_lumps(force_solve=False, method=method)

        return lumps

In [ ]:
# Find the lumps for each carbon source to CO2
lumps = {}

In [ ]:
carbon_source = "EX_hdca_e"
n_carbon = 16

# Force ATPs to be active for the respiratory lumps 
recon3d.reactions.get_by_id('ATPS4mi').bounds = (1,200)

lumps[carbon_source] = find_lumps(carbon_source, n_carbon, recon3d, method='min',
                                    core_reactions=core_reaction_ids,
                                    tissue_reaction_ko=tissue_reaction_ko)

2024-08-16 13:35:41,980 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Max CO2 production:  16.000016000000016
Max ATP production:  106.00010599999999
Test feasability:  0.0
Timeout limit is 3600s


2024-08-16 13:38:14,292 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...


Preparing sinks...


2024-08-16 13:38:30,078 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/F
2024-08-16 13:38:30,079 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/U
2024-08-16 13:38:30,080 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-08-16 13:38:30,082 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-08-16 13:38:30,086 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/F
2024-08-16 13:38:30,088 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/U
2024-08-16 13:38:30,090 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-08-16 13:38:30,092 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-08-16 13:38:30,133 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52H92N3O27FULLRCO/F
2024-08-16 13:38:30,134 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52

Lumping method detected: min


met=co2_e: 100%|██████████| 1/1 [18:33<00:00, 1113.45s/it]


In [ ]:
lumps

{'EX_hdca_e': {<Metabolite co2_e at 0x1ee90c71190>: [Lump(id_='LUMP_EX_co2_e_co2_e', metabolites=defaultdict(<class 'int'>, {'h_m': 8.6875, 'nad_m': -1.4375, 'nadh_m': 1.4375, 'adp_m': 6.25, 'atp_m': -6.25, 'h2o_m': -9.125, 'oaa_m': -0.5, 'fad_m': -0.4375, 'fadh2_m': 0.4375, 'hdca_e': -0.0625, 'fum_m': -0.5, 'mal_L_m': 0.5, 'o2_m': 1.4375, 'pi_m': 6.25, 'succ_m': 0.5, 'co2_e': 1.0, 'h2o_e': 1.0, 'h_e': -0.0625, 'o2_e': -1.4375}), subnetwork={'ACONTm': 0.5, 'AKGDm': 0.5, 'ATPtm': 6.75, 'C160CPT1': 0.0625, 'C160CPT2': 0.0625, 'CO2tm': -1.0, 'CSm': 0.5, 'FACOAL160i': 0.0625, 'FAOXC160': 0.0625, 'FATP1t': -0.0625, 'FUMm': 0.5, 'H2Otm': -7.6874999999999964, 'ICDHxm': 0.5, 'NDPK1m': -0.5, 'O2tm': 1.4374999999999993, 'SUCOAS1m': -0.5, 'r2435': 0.0625, 'ADK1': 0.0625, 'CO2t': -1.0, 'H2Ot': -0.9999999999999972, 'NAt3_1': 0.0625000000000001, 'O2t': 1.4374999999999993, 'PPA': 0.0625, 'cyt_atp2adp': 6.625, 'PiC': 6.75}, gene_reaction_rule='(50.1 or 48.1) and ((4967.2 and 1738.1 and 8050.1 and 1743

In [ ]:
# Compute a lump for glucose respiration
carbon_source = 'EX_glc_D_e'
n_carbon = 6

recon3d.reactions.get_by_id('ATPS4mi').bounds = (1,200)

# Add glycolysis lumps
lumps[carbon_source+'_ox'] = find_lumps(carbon_source, n_carbon, recon3d, method='min',
                                   core_reactions=core_reaction_ids,
                                    tissue_reaction_ko=tissue_reaction_ko)

2024-08-16 13:58:36,877 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Max CO2 production:  6.000006000000001
Max ATP production:  32.000032000000076
Test feasability:  0.0
Timeout limit is 3600s


2024-08-16 14:00:50,605 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...


Preparing sinks...


2024-08-16 14:01:06,259 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/F
2024-08-16 14:01:06,260 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/U
2024-08-16 14:01:06,262 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-08-16 14:01:06,264 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-08-16 14:01:06,266 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/F
2024-08-16 14:01:06,268 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/U
2024-08-16 14:01:06,269 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-08-16 14:01:06,271 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-08-16 14:01:06,308 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52H92N3O27FULLRCO/F
2024-08-16 14:01:06,310 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52

Lumping method detected: min


met=co2_e: 100%|██████████| 1/1 [13:02<00:00, 782.48s/it]


In [ ]:
lumps

{'EX_hdca_e': {<Metabolite co2_e at 0x1ee90c71190>: [Lump(id_='LUMP_EX_co2_e_co2_e', metabolites=defaultdict(<class 'int'>, {'h_m': 8.6875, 'nad_m': -1.4375, 'nadh_m': 1.4375, 'adp_m': 6.25, 'atp_m': -6.25, 'h2o_m': -9.125, 'oaa_m': -0.5, 'fad_m': -0.4375, 'fadh2_m': 0.4375, 'hdca_e': -0.0625, 'fum_m': -0.5, 'mal_L_m': 0.5, 'o2_m': 1.4375, 'pi_m': 6.25, 'succ_m': 0.5, 'co2_e': 1.0, 'h2o_e': 1.0, 'h_e': -0.0625, 'o2_e': -1.4375}), subnetwork={'ACONTm': 0.5, 'AKGDm': 0.5, 'ATPtm': 6.75, 'C160CPT1': 0.0625, 'C160CPT2': 0.0625, 'CO2tm': -1.0, 'CSm': 0.5, 'FACOAL160i': 0.0625, 'FAOXC160': 0.0625, 'FATP1t': -0.0625, 'FUMm': 0.5, 'H2Otm': -7.6874999999999964, 'ICDHxm': 0.5, 'NDPK1m': -0.5, 'O2tm': 1.4374999999999993, 'SUCOAS1m': -0.5, 'r2435': 0.0625, 'ADK1': 0.0625, 'CO2t': -1.0, 'H2Ot': -0.9999999999999972, 'NAt3_1': 0.0625000000000001, 'O2t': 1.4374999999999993, 'PPA': 0.0625, 'cyt_atp2adp': 6.625, 'PiC': 6.75}, gene_reaction_rule='(50.1 or 48.1) and ((4967.2 and 1738.1 and 8050.1 and 1743

In [ ]:
# Compute a lump for glycolysis (Direct respiration makes no sense ... )
carbon_source = 'EX_glc_D_e'
n_carbon = 2
output = 'EX_lac_L_e'

# No need
recon3d.reactions.get_by_id('ATPS4mi').bounds = (0,200)

# Add glycolysis lumps
lumps[carbon_source] = find_lumps(carbon_source, n_carbon, recon3d, method='min',
                                   core_reactions=core_reaction_ids, output=output,
                                    tissue_reaction_ko=tissue_reaction_ko)


2024-08-16 14:15:39,445 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Max CO2 production:  2.000001999999995
Max ATP production:  2.000047000000143
Test feasability:  0.0
Timeout limit is 3600s


2024-08-16 14:17:51,789 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...


Preparing sinks...


2024-08-16 14:18:06,832 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/F
2024-08-16 14:18:06,834 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/U
2024-08-16 14:18:06,835 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-08-16 14:18:06,837 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-08-16 14:18:06,841 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/F
2024-08-16 14:18:06,843 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/U
2024-08-16 14:18:06,846 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-08-16 14:18:06,848 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-08-16 14:18:06,870 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52H92N3O27FULLRCO/F
2024-08-16 14:18:06,871 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52

Lumping method detected: min


met=lac_L_e: 100%|██████████| 1/1 [02:01<00:00, 121.29s/it]


In [ ]:
carbon_source = "EX_lac_L_e"
n_carbon = 3
output = 'EX_co2_e'

# Force ATPs to be active for the respiratory lumps
recon3d.reactions.get_by_id('ATPS4mi').bounds = (1,200)

lumps[carbon_source] = find_lumps(carbon_source, n_carbon, recon3d, method='min',
                                    core_reactions=core_reaction_ids,
                                    tissue_reaction_ko=tissue_reaction_ko)

2024-08-16 14:21:34,861 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Max CO2 production:  3.0000029999998787
Max ATP production:  15.000014999999966
Test feasability:  0.0
Timeout limit is 3600s


2024-08-16 14:23:53,527 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...


Preparing sinks...


2024-08-16 14:24:08,331 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/F
2024-08-16 14:24:08,332 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/U
2024-08-16 14:24:08,334 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-08-16 14:24:08,336 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-08-16 14:24:08,340 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/F
2024-08-16 14:24:08,342 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/U
2024-08-16 14:24:08,344 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-08-16 14:24:08,345 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-08-16 14:24:08,391 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52H92N3O27FULLRCO/F
2024-08-16 14:24:08,394 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52

Lumping method detected: min


met=co2_e: 100%|██████████| 1/1 [47:18<00:00, 2838.64s/it]


In [ ]:
lumps

{'EX_hdca_e': {<Metabolite co2_e at 0x1ee90c71190>: [Lump(id_='LUMP_EX_co2_e_co2_e', metabolites=defaultdict(<class 'int'>, {'h_m': 8.6875, 'nad_m': -1.4375, 'nadh_m': 1.4375, 'adp_m': 6.25, 'atp_m': -6.25, 'h2o_m': -9.125, 'oaa_m': -0.5, 'fad_m': -0.4375, 'fadh2_m': 0.4375, 'hdca_e': -0.0625, 'fum_m': -0.5, 'mal_L_m': 0.5, 'o2_m': 1.4375, 'pi_m': 6.25, 'succ_m': 0.5, 'co2_e': 1.0, 'h2o_e': 1.0, 'h_e': -0.0625, 'o2_e': -1.4375}), subnetwork={'ACONTm': 0.5, 'AKGDm': 0.5, 'ATPtm': 6.75, 'C160CPT1': 0.0625, 'C160CPT2': 0.0625, 'CO2tm': -1.0, 'CSm': 0.5, 'FACOAL160i': 0.0625, 'FAOXC160': 0.0625, 'FATP1t': -0.0625, 'FUMm': 0.5, 'H2Otm': -7.6874999999999964, 'ICDHxm': 0.5, 'NDPK1m': -0.5, 'O2tm': 1.4374999999999993, 'SUCOAS1m': -0.5, 'r2435': 0.0625, 'ADK1': 0.0625, 'CO2t': -1.0, 'H2Ot': -0.9999999999999972, 'NAt3_1': 0.0625000000000001, 'O2t': 1.4374999999999993, 'PPA': 0.0625, 'cyt_atp2adp': 6.625, 'PiC': 6.75}, gene_reaction_rule='(50.1 or 48.1) and ((4967.2 and 1738.1 and 8050.1 and 1743

In [ ]:
carbon_source = "EX_bhb_e"
n_carbon = 4

# Force ATPs to be active for the respiratory lumps 
recon3d.reactions.get_by_id('ATPS4mi').bounds = (1,200)

lumps[carbon_source] = find_lumps(carbon_source, n_carbon, recon3d, method='min',
                                    core_reactions=core_reaction_ids,
                                    tissue_reaction_ko=tissue_reaction_ko)

2024-08-16 15:12:55,848 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Max CO2 production:  4.000004
Max ATP production:  21.50002149999989
Test feasability:  0.0
Timeout limit is 3600s


2024-08-16 15:15:20,499 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...


Preparing sinks...


2024-08-16 15:15:36,718 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/F
2024-08-16 15:15:36,720 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/U
2024-08-16 15:15:36,720 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-08-16 15:15:36,722 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-08-16 15:15:36,724 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/F
2024-08-16 15:15:36,727 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/U
2024-08-16 15:15:36,730 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-08-16 15:15:36,732 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-08-16 15:15:36,770 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52H92N3O27FULLRCO/F
2024-08-16 15:15:36,771 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52

Lumping method detected: min


met=co2_e: 100%|██████████| 1/1 [10:00<00:00, 600.40s/it]


In [ ]:
# Save the lumps in a json file Lumps are NamedTuple objects
import json

lumps_json = {k: [i for m,l in v.items() for i in l ] for k,v in lumps.items()}
with open('lumps.json', 'w') as f:
    json.dump(lumps_json, f, indent=4)


In [ ]:
lumps_json

{'EX_hdca_e': [Lump(id_='LUMP_EX_co2_e_co2_e', metabolites=defaultdict(<class 'int'>, {'h_m': 8.6875, 'nad_m': -1.4375, 'nadh_m': 1.4375, 'adp_m': 6.25, 'atp_m': -6.25, 'h2o_m': -9.125, 'oaa_m': -0.5, 'fad_m': -0.4375, 'fadh2_m': 0.4375, 'hdca_e': -0.0625, 'fum_m': -0.5, 'mal_L_m': 0.5, 'o2_m': 1.4375, 'pi_m': 6.25, 'succ_m': 0.5, 'co2_e': 1.0, 'h2o_e': 1.0, 'h_e': -0.0625, 'o2_e': -1.4375}), subnetwork={'ACONTm': 0.5, 'AKGDm': 0.5, 'ATPtm': 6.75, 'C160CPT1': 0.0625, 'C160CPT2': 0.0625, 'CO2tm': -1.0, 'CSm': 0.5, 'FACOAL160i': 0.0625, 'FAOXC160': 0.0625, 'FATP1t': -0.0625, 'FUMm': 0.5, 'H2Otm': -7.6874999999999964, 'ICDHxm': 0.5, 'NDPK1m': -0.5, 'O2tm': 1.4374999999999993, 'SUCOAS1m': -0.5, 'r2435': 0.0625, 'ADK1': 0.0625, 'CO2t': -1.0, 'H2Ot': -0.9999999999999972, 'NAt3_1': 0.0625000000000001, 'O2t': 1.4374999999999993, 'PPA': 0.0625, 'cyt_atp2adp': 6.625, 'PiC': 6.75}, gene_reaction_rule='(50.1 or 48.1) and ((4967.2 and 1738.1 and 8050.1 and 1743.1) or (4967.1 and 1738.1 and 8050.1 a

In [ ]:
# Make a new model with the lumps only containing the core reactions and the lumps
reduced_model = recon3d.copy()

2024-08-16 15:27:10,182 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


In [ ]:
boundary = [r.id for r in reduced_model.reactions if r in reduced_model.boundary]
lump_subnet_reactions = list({r for lumps in lumps_json.values() for l in lumps for r in l.subnetwork})
reduced_reactions = core_reaction_ids + lump_subnet_reactions + boundary

reactions_to_remove = [r for r in reduced_model.reactions if r.id not in reduced_reactions]
reduced_model.remove_reactions(reactions_to_remove)



In [ ]:
reduced_model.prepare()
reduced_model.convert()


2024-08-16 15:40:25,078 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...
2024-08-16 15:40:40,041 - thermomodel_Recon3thermoCurated - INFO - # Model preparation done.
2024-08-16 15:40:40,043 - thermomodel_Recon3thermoCurated - INFO - # Model conversion starting...
2024-08-16 15:40:59,084 - thermomodel_Recon3thermoCurated - INFO - # Model conversion done.
2024-08-16 15:40:59,086 - thermomodel_Recon3thermoCurated - INFO - # Updating cobra_model variables...
2024-08-16 15:40:59,206 - thermomodel_Recon3thermoCurated - INFO - # cobra_model variables are up-to-date


In [ ]:
# Make flux variability and remove reactions that are not active
reduced_model.objective = reduced_model.reactions.get_by_id('cyt_atp2adp') 

carbon_sources = ['EX_glc_D_e', 'EX_lac_L_e', 'EX_bhb_e', 'EX_hdca_e']

for carbon_source in carbon_sources:
    reduced_model.reactions.get_by_id(carbon_source).bounds = (-1, 0)


reduced_model.optimize()

,fluxes,reduced_costs
ACACT1rm,-1.000000,None
ACONTm,12.861111,None
AKGDm,12.861111,None
AKGMALtm,0.000000,None
ASPGLUm,0.000000,None
...,...,...
NADH2_u10mi,52.305556,None
CYOOm3i,0.000000,None
CYOOm2i,36.083333,None
cyt_atp2adp,172.416667,None


In [ ]:
reduced_model.medium

{'EX_bhb_e': 1,
 'EX_hdca_e': 1,
 'EX_h_e': 200.0,
 'EX_h2o_e': 200.0,
 'EX_lac_L_e': 1,
 'EX_o2_e': 200.0,
 'EX_glc_D_e': 1}

In [ ]:
# TVA to remove reactions that are not active
from pytfa.analysis import variability_analysis

FVA = variability_analysis(reduced_model, kind='reactions')


2024-08-16 15:40:59,796 - thermomodel_Recon3thermoCurated - INFO - Beginning variability analysis for variable of type reactions
maximizing: 100%|██████████| 1886/1886 [07:15<00:00,  4.33it/s]


In [ ]:
EPSILON = 1e-9

reactions_to_remove = [r for r in reduced_model.reactions if FVA.loc[r.id, 'minimum'] >= -EPSILON and FVA.loc[r.id, 'maximum'] <= EPSILON ]
len(reactions_to_remove)

1815

In [ ]:
# Load model for sake of time
# from pytfa.io.json import load_json_model
# reduced_model = load_json_model('reduced_model_no_core_20240412-090529.json')


EPSILON = 1e-9
# remove reactions that are not active min and max are zero
reactions_to_remove = [r for r in reduced_model.reactions if FVA.loc[r.id, 'minimum'] >= -EPSILON and FVA.loc[r.id, 'maximum'] <= EPSILON ]
reduced_model.remove_reactions(reactions_to_remove)
reduced_model.repair()
reduced_model.optimize()

metabolites_to_remove = [m for m in reduced_model.metabolites if len(m.reactions) == 0]
reduced_model.remove_metabolites(metabolites_to_remove)


In [ ]:
reduced_model.repair()
reduced_model.optimize()

,fluxes,reduced_costs
ACACT1rm,-1.000000,None
ACONTm,12.861111,None
AKGDm,12.861111,None
AKGMALtm,0.000000,None
ASPGLUm,0.000000,None
...,...,...
CYOR_u10mi,72.166667,None
NADH2_u10mi,52.305556,None
CYOOm2i,36.083333,None
cyt_atp2adp,172.416667,None


In [ ]:
from pytfa.analysis import variability_analysis
FVA = variability_analysis(reduced_model, kind='reactions')
# Show the remaning reactions

2024-08-16 15:52:14,265 - thermomodel_Recon3thermoCurated - INFO - Beginning variability analysis for variable of type reactions
maximizing: 100%|██████████| 71/71 [00:04<00:00, 15.08it/s]


In [ ]:
# Concentration ranges
from pytfa.optim.variables import LogConcentration
TVA = variability_analysis(reduced_model, kind=LogConcentration)

2024-08-16 15:52:23,304 - thermomodel_Recon3thermoCurated - INFO - Beginning variability analysis for variable of type <class 'pytfa.optim.variables.LogConcentration'>
maximizing: 100%|██████████| 83/83 [00:05<00:00, 14.51it/s]


In [ ]:
from pytfa.io.json import save_json_model
import datetime 
# Save the reduced model as json
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
reduced_model.id = 'reduced_model_ETC_core_{}'.format(timestamp)

save_json_model(reduced_model, 'reduced_model_ETC_core_{}.json'.format(timestamp))


In [ ]:
from cobra.io.json import save_json_model as cobra_save_json_model

cobra_save_json_model(reduced_model, 'reduced_model_ETC_core_fba_only_{}.json'.format(timestamp))

In [ ]:
# from cobra.io.json import save_json_model as cobra_save_json_model

# cobra_save_json_model(recon3d, 'recon3d_fba_only.json')